In [ ]:
import scipy
import numpy as np
import panel as pn

import holoviews as hv
import pandas as pd

import holoviews.plotting.bokeh
from bokeh.tile_providers import get_provider, Vendors
from bokeh.plotting import figure, show, output_file
from bokeh.models import (
    Slider,
    CheckboxGroup,
    CustomJS,
    ColumnDataSource,
    CDSView,
    Scatter,
    GeoJSONDataSource,
    Axis,
    LinearAxis,
    BoxZoomTool,
    ResetTool,
    PanTool,
    Div,
    LinearColorMapper,
)
from bokeh.models.filters import CustomJSFilter
from bokeh.palettes import brewer

In [ ]:
pn.extension()


def get_coastlines():
    COASTLINES = scipy.io.loadmat("coastlines.mat")
    COASTLINES["lon"] = COASTLINES["lon"].flatten()
    COASTLINES["lat"] = COASTLINES["lat"].flatten()
    return COASTLINES


COASTLINES = get_coastlines()

plot = figure(
    x_range=(0, 360),
    y_range=(-90, 90),
    width=800,
    height=400,
    match_aspect=True,
    # tools=[BoxZoomTool(), ResetTool(), PanTool()],
    # tools=[BoxZoomTool(match_aspect=True), ResetTool(), PanTool()],
    output_backend="webgl",
    toolbar_location="below",
)

plot.line(
    COASTLINES["lon"],
    COASTLINES["lat"],
    color="black",
    line_width=0.5,
)

plot.xgrid.visible = False
plot.ygrid.visible = False
plot.add_layout(LinearAxis(), "above")  # Add axis on the top
plot.add_layout(LinearAxis(), "right")  # Add axis on the right

callback = CustomJS(
    args=dict(plot=plot),
    code="""
    var x_range = plot.x_range.end - plot.x_range.start;
    var y_range = plot.y_range.end - plot.y_range.start;

    // Calculate the aspect ratio of the zoomed-in area
    var aspect_ratio = x_range / y_range;

    // Adjust the width to maintain the aspect ratio while keeping height constant
    var new_width = plot.height * aspect_ratio;
    var new_height = 400;

    // Apply the new width
    plot.width = new_width;

    // If width >= 800 then shrink height
    if (new_width >= 800) {
        new_width = 800;
        new_height = new_width / aspect_ratio;
    }

    // Package result
    plot.width = new_width;
    plot.height = new_height;
""",
)

# Attach the callback to the 'x_range' and 'y_range' change events
plot.x_range.js_on_change("start", callback)
plot.x_range.js_on_change("end", callback)
plot.y_range.js_on_change("start", callback)
plot.y_range.js_on_change("end", callback)

# Step 5: Display the plot
output_file("zoom_reset.html")

show(plot)

In [ ]:
# Set up data set
station_file_1 = "model_station2.csv"
station = pd.read_csv(station_file_1)


def get_coastlines():
    COASTLINES = scipy.io.loadmat("coastlines.mat")
    COASTLINES["lon"] = COASTLINES["lon"].flatten()
    COASTLINES["lat"] = COASTLINES["lat"].flatten()
    return COASTLINES


COASTLINES = get_coastlines()


VELOCITY_SCALE = 0.01
source = ColumnDataSource(
    data={
        "lon": station2.lon,
        "lat": station2.lat,
        "obs_east_vel": station2.east_vel,
        "obs_north_vel": station2.north_vel,
        "obs_east_vel_lon": station2.lon + VELOCITY_SCALE * station2.east_vel,
        "obs_north_vel_lat": station2.lat + VELOCITY_SCALE * station2.north_vel,
        "mod_east_vel": station2.model_east_vel,
        "mod_north_vel": station2.model_north_vel,
        "mod_east_vel_lon": station2.lon + VELOCITY_SCALE * station2.model_east_vel,
        "mod_north_vel_lat": station2.lat + VELOCITY_SCALE * station2.model_north_vel,
    }
)

################
# Figure setup #
################
fig = figure(
    x_range=(0, 360),
    y_range=(-90, 90),
    width=800,
    height=400,
    match_aspect=True,
    tools=[BoxZoomTool(match_aspect=True), ResetTool(), PanTool()],
    output_backend="webgl",
)
fig.xgrid.visible = False
fig.ygrid.visible = False
fig.add_layout(LinearAxis(), "above")  # Add axis on the top
fig.add_layout(LinearAxis(), "right")  # Add axis on the right


# Grid layout
grid_layout = pn.GridSpec(sizing_mode="stretch_both", max_height=600)


##############
# UI objects #
##############
# Folder 1 controls
folder_name_1 = "0000000292"
folder_label_1 = Div(text="folder_1: 0000000292")
loc_checkbox_1 = CheckboxGroup(labels=["locs"], active=[])
name_checkbox_1 = CheckboxGroup(labels=["name"], active=[])
obs_vel_checkbox_1 = CheckboxGroup(labels=["obs"], active=[])
mod_vel_checkbox_1 = CheckboxGroup(labels=["mod"], active=[])
res_vel_checkbox_1 = CheckboxGroup(labels=["res"], active=[])
rot_vel_checkbox_1 = CheckboxGroup(labels=["rot"], active=[])
seg_vel_checkbox_1 = CheckboxGroup(labels=["seg"], active=[])
tri_vel_checkbox_1 = CheckboxGroup(labels=["tri"], active=[])
str_vel_checkbox_1 = CheckboxGroup(labels=["str"], active=[])
mog_vel_checkbox_1 = CheckboxGroup(labels=["mog"], active=[])

# Folder 2 controls
folder_name_2 = "0000000293"
folder_label_2 = Div(text="folder_2: 0000000293")
loc_checkbox_2 = CheckboxGroup(labels=["locs"], active=[])
name_checkbox_2 = CheckboxGroup(labels=["name"], active=[])
obs_vel_checkbox_2 = CheckboxGroup(labels=["obs"], active=[])
mod_vel_checkbox_2 = CheckboxGroup(labels=["mod"], active=[])
res_vel_checkbox_2 = CheckboxGroup(labels=["res"], active=[])
rot_vel_checkbox_2 = CheckboxGroup(labels=["rot"], active=[])
seg_vel_checkbox_2 = CheckboxGroup(labels=["seg"], active=[])
tri_vel_checkbox_2 = CheckboxGroup(labels=["tri"], active=[])
str_vel_checkbox_2 = CheckboxGroup(labels=["str"], active=[])
mog_vel_checkbox_2 = CheckboxGroup(labels=["mog"], active=[])

# Other controls
velocity_scaler = Slider(
    start=0.0, end=50, value=1, step=1.0, title="vel scale", width=200
)


###############
# Map objects #
###############

# Coastlines
fig.line(
    COASTLINES["lon"],
    COASTLINES["lat"],
    color="black",
    line_width=0.5,
)

# Create glyphs all potential plotting elements and hide them as default
loc_obj_1 = fig.scatter(
    "lon", "lat", source=source, size=1, color="black", visible=False
)

# Folder 1: observed velocities
obs_vel_obj_1 = fig.segment(
    "lon",
    "lat",
    "obs_east_vel_lon",
    "obs_north_vel_lat",
    source=source,
    line_width=1,
    color="blue",
    alpha=0.5,
    visible=False,
)

mod_vel_obj_1 = fig.segment(
    "lon",
    "lat",
    "mod_east_vel_lon",
    "mod_north_vel_lat",
    source=source,
    line_width=1,
    color="red",
    alpha=0.5,
    visible=False,
)


#############
# Callbacks #
#############
# Define JavaScript callbacks to toggle visibility
loc_checkbox_callback_1 = CustomJS(
    args={"scatter": loc_obj_1},
    code="""
    scatter.visible = cb_obj.active.includes(0);
""",
)

obs_vel_checkbox_callback_1 = CustomJS(
    args={"segment": obs_vel_obj_1},
    code="""
    segment.visible = cb_obj.active.includes(0);
""",
)

mod_vel_checkbox_callback_1 = CustomJS(
    args={"segment": mod_vel_obj_1},
    code="""
    segment.visible = cb_obj.active.includes(0);
""",
)


# JavaScript callback for velocity magnitude scaling
velocity_scaler_callback = CustomJS(
    args=dict(
        source=source, velocity_scaler=velocity_scaler, VELOCITY_SCALE=VELOCITY_SCALE
    ),
    code="""
    const velocity_scale_slider = velocity_scaler.value
    const lon = source.data.lon
    const lat = source.data.lat
    const obs_east_vel = source.data.obs_east_vel
    const obs_north_vel = source.data.obs_north_vel
    const mod_east_vel = source.data.mod_east_vel
    const mod_north_vel = source.data.mod_north_vel

    // Update velocities with current magnitude scaling
    let obs_east_vel_lon = [];
    let obs_north_vel_lat = [];
    let mod_east_vel_lon = [];
    let mod_north_vel_lat = [];
    for (let i = 0; i < lon.length; i++) {
        obs_east_vel_lon.push(lon[i] + VELOCITY_SCALE * velocity_scale_slider * obs_east_vel[i]);
        obs_north_vel_lat.push(lat[i] + VELOCITY_SCALE * velocity_scale_slider * obs_north_vel[i]);
        mod_east_vel_lon.push(lon[i] + VELOCITY_SCALE * velocity_scale_slider * mod_east_vel[i]);
        mod_north_vel_lat.push(lat[i] + VELOCITY_SCALE * velocity_scale_slider * mod_north_vel[i]);
    }

    // Package everthing back into dictionary
    // Try source.change.emit();???
    source.data = { lon, lat, obs_east_vel, obs_north_vel, obs_east_vel_lon, obs_north_vel_lat, mod_east_vel, mod_north_vel, mod_east_vel_lon, mod_north_vel_lat }
""",
)


###################################
# Attach the callbacks to handles #
###################################
loc_checkbox_1.js_on_change("active", loc_checkbox_callback_1)
obs_vel_checkbox_1.js_on_change("active", obs_vel_checkbox_callback_1)
mod_vel_checkbox_1.js_on_change("active", mod_vel_checkbox_callback_1)
velocity_scaler.js_on_change("value", velocity_scaler_callback)


##############################
# Place objects on panel grid #
###############################
# Placing controls for folder 1
grid_layout[0:1, 0] = pn.Column(
    pn.pane.Bokeh(folder_label_1),
    pn.pane.Bokeh(loc_checkbox_1),
    pn.pane.Bokeh(obs_vel_checkbox_1),
    pn.pane.Bokeh(mod_vel_checkbox_1),
    pn.pane.Bokeh(res_vel_checkbox_1),
    pn.pane.Bokeh(rot_vel_checkbox_1),
    pn.pane.Bokeh(seg_vel_checkbox_1),
    pn.pane.Bokeh(tri_vel_checkbox_1),
    pn.pane.Bokeh(str_vel_checkbox_1),
    pn.pane.Bokeh(mog_vel_checkbox_1),
)

grid_layout[0:1, 1] = pn.Column(
    pn.pane.Bokeh(folder_label_2),
    pn.pane.Bokeh(loc_checkbox_2),
    pn.pane.Bokeh(obs_vel_checkbox_2),
    pn.pane.Bokeh(mod_vel_checkbox_2),
    pn.pane.Bokeh(res_vel_checkbox_2),
    pn.pane.Bokeh(rot_vel_checkbox_2),
    pn.pane.Bokeh(seg_vel_checkbox_2),
    pn.pane.Bokeh(tri_vel_checkbox_2),
    pn.pane.Bokeh(str_vel_checkbox_2),
    pn.pane.Bokeh(mog_vel_checkbox_2),
)

grid_layout[4:6, 0:1] = pn.Column(
    pn.pane.Bokeh(velocity_scaler),
)


# Place map
grid_layout[0:8, 2:10] = fig

# Plot the grid layout
grid_layout

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CheckboxGroup, CustomJS
from bokeh.layouts import column


VELOCITY_SCALE = 0.01


station_file_1 = "model_station2.csv"
station = pd.read_csv(station_file_1)

source = ColumnDataSource(
    data={
        "lon": station2.lon,
        "lat": station2.lat,
        "obs_east_vel": station2.east_vel,
        "obs_north_vel": station2.north_vel,
        "obs_east_vel_lon": station2.lon + VELOCITY_SCALE * station2.east_vel,
        "obs_north_vel_lat": station2.lat + VELOCITY_SCALE * station2.north_vel,
        "mod_east_vel": station2.model_east_vel,
        "mod_north_vel": station2.model_north_vel,
        "mod_east_vel_lon": station2.lon + VELOCITY_SCALE * station2.model_east_vel,
        "mod_north_vel_lat": station2.lat + VELOCITY_SCALE * station2.model_north_vel,
    }
)

# Create a figure
p = figure(match_aspect=True, tools=[BoxZoomTool(match_aspect=True)])
p.xgrid.visible = False
p.ygrid.visible = False
p.add_layout(LinearAxis(), "above")  # Add axis on the top
p.add_layout(LinearAxis(), "right")  # Add axis on the right

# Create glyphs all potential plotting elements and hide them as default
locs_handle = p.scatter(
    "lon", "lat", source=source, size=1, color="black", visible=False
)

obs_vel_handle = p.segment(
    "lon",
    "lat",
    "obs_east_vel_lon",
    "obs_north_vel_lat",
    source=source,
    line_width=1,
    color="blue",
    alpha=0.5,
    visible=False,
)

mod_vel_handle = p.segment(
    "lon",
    "lat",
    "mod_east_vel_lon",
    "mod_north_vel_lat",
    source=source,
    line_width=1,
    color="red",
    alpha=0.5,
    visible=False,
)

# Create checkboxes for toggling visibility, initially unchecked
locs_checkbox = CheckboxGroup(labels=["locs"], active=[])
obs_vel_checkbox = CheckboxGroup(labels=["obs"], active=[])
mod_vel_checkbox = CheckboxGroup(labels=["mod"], active=[])

# Slider with the intention of scaling the magnitude of segments
velocity_scaler = Slider(
    start=0.0, end=10, value=1, step=0.1, title="vel scale", width=100
)

# Define JavaScript callbacks to toggle visibility
locs_checkbox_callback = CustomJS(
    args={"scatter": locs_handle},
    code="""
    scatter.visible = cb_obj.active.includes(0);
""",
)

obs_vel_checkbox_callback = CustomJS(
    args={"segment": obs_vel_handle},
    code="""
    segment.visible = cb_obj.active.includes(0);
""",
)

mod_vel_checkbox_callback = CustomJS(
    args={"segment": mod_vel_handle},
    code="""
    segment.visible = cb_obj.active.includes(0);
""",
)

# JavaScript callback for velocity magnitude scaling
velocity_scaler_callback = CustomJS(
    args=dict(
        source=source, velocity_scaler=velocity_scaler, VELOCITY_SCALE=VELOCITY_SCALE
    ),
    code="""
    const velocity_scale_slider = velocity_scaler.value
    const lon = source.data.lon
    const lat = source.data.lat
    const obs_east_vel = source.data.obs_east_vel
    const obs_north_vel = source.data.obs_north_vel
    const mod_east_vel = source.data.mod_east_vel
    const mod_north_vel = source.data.mod_north_vel

    // Update velocities with current magnitude scaling
    let obs_east_vel_lon = [];
    let obs_north_vel_lat = [];
    let mod_east_vel_lon = [];
    let mod_north_vel_lat = [];
    for (let i = 0; i < lon.length; i++) {
        obs_east_vel_lon.push(lon[i] + VELOCITY_SCALE * velocity_scale_slider * obs_east_vel[i]);
        obs_north_vel_lat.push(lat[i] + VELOCITY_SCALE * velocity_scale_slider * obs_north_vel[i]);
        mod_east_vel_lon.push(lon[i] + VELOCITY_SCALE * velocity_scale_slider * mod_east_vel[i]);
        mod_north_vel_lat.push(lat[i] + VELOCITY_SCALE * velocity_scale_slider * mod_north_vel[i]);
    }

    // Package everthing back into dictionary
    // Try source.change.emit();???
    source.data = { lon, lat, obs_east_vel, obs_north_vel, obs_east_vel_lon, obs_north_vel_lat, mod_east_vel, mod_north_vel, mod_east_vel_lon, mod_north_vel_lat }
""",
)

# Attach the callbacks to handles
locs_checkbox.js_on_change("active", locs_checkbox_callback)
obs_vel_checkbox.js_on_change("active", obs_vel_checkbox_callback)
mod_vel_checkbox.js_on_change("active", mod_vel_checkbox_callback)
velocity_scaler.js_on_change("value", velocity_scaler_callback)

# Layout the plot and the checkboxes
layout = column(locs_checkbox, obs_vel_checkbox, mod_vel_checkbox, velocity_scaler, p)

# Display the plot
show(layout)

# Getting a filename

In [ ]:
from bokeh.models import CustomJS, Button, Div
from bokeh.layouts import column
from bokeh.plotting import show
from bokeh.io import output_file

# Step 1: Create a Bokeh button
button = Button(label="Select a File", button_type="success")

# Step 2: Create a Div to display the selected file name
div = Div(text="No file selected", width=400, height=100)

# Step 3: JavaScript code to create a hidden file input and handle the file selection
callback = CustomJS(
    args=dict(div=div),
    code="""
    var input = document.createElement('input');
    input.type = 'file';
    input.onchange = function() {
        var file = input.files[0];
        if (file) {
            div.text = 'Selected file: ' + file.name;
        } else {
            div.text = 'No file selected';
        }
    };
    input.click();
""",
)

# Step 4: Attach the callback to the button
button.js_on_event("button_click", callback)

# Step 5: Layout and show
layout = column(button, div)
# output_file("file_select.html")
show(layout)

# Plotting an image with axis limits

In [ ]:
# import numpy as np
# from PIL import Image
# from bokeh.plotting import figure, show
# from bokeh.io import output_file

# # Step 1: Load the image and convert to a NumPy array
# image_path = "GRAY_50M_SR_OB.jpg"
# image = Image.open(image_path).convert("RGBA")
# image_data = np.array(image)

# # Step 2: Prepare the image data for Bokeh
# # Flip the image vertically and reshape the data
# image_data = np.flipud(image_data)  # Flip the image to match Bokeh's coordinate system
# image_data = image_data.view(np.uint32).reshape((5400, 10800))
# # 10800 × 5400
# # Step 3: Create a Bokeh plot
# p = figure(x_range=(0, 10), y_range=(0, 10), width=400, height=400)

# # Step 4: Plot the image on the Bokeh figure
# p.image_rgba(image=[image_data], x=0, y=0, dw=10, dh=10)

# # Optional: Set axis labels
# p.xaxis.axis_label = "X Axis"
# p.yaxis.axis_label = "Y Axis"

# # Step 5: Display the plot
# output_file("image_plot.html")
# show(p)

In [ ]:
from bokeh.models import CustomJS, Button, Div
from bokeh.layouts import column
from bokeh.plotting import show
from bokeh.io import output_file

# Step 1: Create a Bokeh button
button = Button(label="Select a Folder", button_type="success")

# Step 2: Create a Div to display the selected folder name
div = Div(text="No folder selected", width=400, height=100)

# Step 3: JavaScript code to create a hidden folder input and handle the folder selection
callback = CustomJS(
    args=dict(div=div),
    code="""
    var input = document.createElement('input');
    input.type = 'file';
    input.webkitdirectory = true;
    input.onchange = function() {
        var files = input.files;
        if (files.length > 0) {
            // Extract the folder path from the first selected file
            var path = files[0].webkitRelativePath;
            var folderName = path.split('/')[0];
            div.text = 'Selected folder: ' + folderName;
        } else {
            div.text = 'No folder selected';
        }
    };
    input.click();
""",
)

# Step 4: Attach the callback to the button
button.js_on_event("button_click", callback)

# Step 5: Layout and show
layout = column(button, div)
# output_file("folder_select.html")
show(layout)

### Comparing residuals from two runs

Modified files so that `0000000343` has all zero north residuals, and is missing the first station2. `0000000344` is missing the second station2.

In [ ]:
import numpy as np

def intersect2D(Array_A, Array_B):
  """
  Find row intersection between 2D numpy arrays, a and b.
  Returns another numpy array with shared rows and index of items in A & B arrays
  """
  for i in range(len(Array_A)):
    

  return intersectionList, idx, idy

In [95]:
folder_name = "0000000344"
station1 = pd.read_csv(folder_name + "/model_station.csv")
stasource_1 = ColumnDataSource(data = {
        "lon_1": station1.lon,
        "lat_1": station1.lat,
        "obs_east_vel_1": station1.east_vel,
        "obs_north_vel_1": station1.north_vel,
        "mod_east_vel_1": station1.model_east_vel,
        "mod_north_vel_1": station1.model_north_vel,
        "res_east_vel_1": station1.model_east_vel_residual,
        "res_north_vel_1": station1.model_north_vel_residual,
        "rot_east_vel_1": station1.model_east_vel_rotation,
        "rot_north_vel_1": station1.model_north_vel_rotation,
        "seg_east_vel_1": station1.model_east_elastic_segment,
        "seg_north_vel_1": station1.model_north_elastic_segment,
        "tde_east_vel_1": station1.model_east_vel_tde,
        "tde_north_vel_1": station1.model_north_vel_tde,
        "str_east_vel_1": station1.model_east_vel_block_strain_rate,
        "str_north_vel_1": station1.model_north_vel_block_strain_rate,
        "mog_east_vel_1": station1.model_east_vel_mogi,
        "mog_north_vel_1": station1.model_north_vel_mogi,
    })
folder_name = "0000000343"
station2 = pd.read_csv(folder_name + "/model_station.csv")
stasource_2 = ColumnDataSource(data = {
        "lon_2": station2.lon,
        "lat_2": station2.lat,
        "obs_east_vel_2": station2.east_vel,
        "obs_north_vel_2": station2.north_vel,
        "mod_east_vel_2": station2.model_east_vel,
        "mod_north_vel_2": station2.model_north_vel,
        "res_east_vel_2": station2.model_east_vel_residual,
        "res_north_vel_2": station2.model_north_vel_residual,
        "rot_east_vel_2": station2.model_east_vel_rotation,
        "rot_north_vel_2": station2.model_north_vel_rotation,
        "seg_east_vel_2": station2.model_east_elastic_segment,
        "seg_north_vel_2": station2.model_north_elastic_segment,
        "tde_east_vel_2": station2.model_east_vel_tde,
        "tde_north_vel_2": station2.model_north_vel_tde,
        "str_east_vel_2": station2.model_east_vel_block_strain_rate,
        "str_north_vel_2": station2.model_north_vel_block_strain_rate,
        "mog_east_vel_2": station2.model_east_vel_mogi,
        "mog_north_vel_2": station2.model_north_vel_mogi,
    })

# Intersect station dataframes based on lon, lat and retain residual velocity components
common = pd.merge(station1, station2, how='inner', on=['lon', 'lat'], suffixes=('_1', '_2'))
not_in_2 = station1[~station1.isin(station2)]
not_in_1 = station2[~station2.isin(station1)]

# Calculate residual magnitudes
common["res_mag_1"] = np.sqrt(common["model_east_vel_residual_1"]**2 + common["model_north_vel_residual_1"]**2)
common["res_mag_2"] = np.sqrt(common["model_east_vel_residual_2"]**2 + common["model_north_vel_residual_2"]**2)
common["res_mag_diff"] = common["res_mag_2"] - common["res_mag_1"]

# ColumnDataSource to hold data for stations common to both folders
commonsta = ColumnDataSource(data = {
        "lon_c": common.lon,
        "lat_c": common.lat,
        "res_mag_diff": common.res_mag_diff,
        "res_mag_diff_sized": 5*np.abs(common.res_mag_diff),})

# coords1 = np.array([stasource_1.data["lon_1"],stasource_1.data["lat_1"]]).T
# coords2 = np.array([stasource_2.data["lon_2"],stasource_2.data["lat_2"]]).T
# # common_coords, idx1, idx2 = intersect2D(coords1, coords2)

# coords1 = np.array([[0, 1], [1, 2], [3, 4]])
# coords2 = np.array([[3, 4], [0, 1], [1, 2], [2, 2], [9, 8]])

random_selection = np.random.rand(10, 2)

coords1 = random_selection[[0, 1, 2, 3], :]
coords2 = random_selection[[5, 3, 1, 0, 7, 8], :]

# Make sure we don't have any lats that equal lons 
coords1[:, 1] += 9999
coords2[:, 1] += 9999

# Cantor pairing of lons, lats
def cantor_pairing(x, y):
    return (x + y) * (x + y + 1) / 2 + y

coord_1 = cantor_pairing(stasource_1.data["lon_1"], stasource_1.data["lat_1"])
coord_2 = cantor_pairing(stasource_2.data["lon_2"], stasource_2.data["lat_2"])

# coord_1 = cantor_pairing(coords1[:, 0], coords1[:, 1])
# coord_2 = cantor_pairing(coords2[:, 0], coords2[:, 1])

# 1D intersection of Cantor pairs
_, index_1, index_2 = np.intersect1d(coord_1, coord_2, return_indices=True)

# Extract common longitudes and latitudes
common_lon = stasource_1.data["lon_1"][index_1]
common_lat = stasource_1.data["lat_1"][index_1]

# Do residual comparison
resmag_1 = np.sqrt(stasource_1.data["res_east_vel_1"][index_1]**2 + stasource_1.data["res_north_vel_1"][index_1]**2)
resmag_2 = np.sqrt(stasource_2.data["res_east_vel_2"][index_2]**2 + stasource_2.data["res_north_vel_2"][index_2]**2)
resmag_d = resmag_2 - resmag_1

# ColumnDataSource to hold data for stations common to both folders
commonsta = ColumnDataSource(data = {
        "lon_c": common_lon,
        "lat_c": common_lat,
        "res_mag_diff": resmag_d,
        "sized_res_mag_diff": 5*np.abs(resmag_d),})

# Setdiffs of Cantor pairs
not_in_2 = np.where(~np.in1d(coord_1,coord_2))[0]
not_in_1 = np.where(~np.in1d(coord_2,coord_1))[0]

# unique_lon = np.concatenate((coords1[not_in_2, 0], coords2[not_in_1, 0]))
# unique_lat = np.concatenate((coords1[not_in_2, 1], coords2[not_in_1, 1]))

unique_lon = np.concatenate((stasource_1.data["lon_1"][not_in_2], stasource_2.data["lon_2"][not_in_1]))
unique_lat = np.concatenate((stasource_1.data["lat_1"][not_in_2], stasource_2.data["lat_2"][not_in_1]))


uniquesta = ColumnDataSource(data = {
        "lon_u": unique_lon,
        "lat_u": unique_lat,})

# common_idx1 = np.isin(coords1, coords2)
# common_idx2 = np.isin(coords2, coords1)

# common1, ci1, ci2 = np.intersect1d(coords1, coords2, return_indices=True)
# # To convert flattened indices to row indices, need to make sure that each even has an odd counterpart
# # Then use even/2 as row indices
# ci1e = ci1[ci1%2==0]
# ci1o = ci1[ci1%2==1]-1
# ci1n, i11n, i12n = np.intersect1d(ci1e, ci1o, return_indices=True)
# row_index1 = ci1n[i11n]/2

# ci2e = ci2[ci2%2==0]
# ci2o = ci2[ci2%2==1]-1
# ci2n, i21n, i22n = np.intersect1d(ci2e, ci2o, return_indices=True)
# row_index2 = ci2n[i21n]/2

# print(coords1.flatten())
# print(coords2.flatten())
# print(common1.reshape(2, -1).T)

# # common_lon, idx1_lon, idx2_lon = np.intersect1d(stasource_1.data["lon_1"], stasource_2.data["lon_2"], return_indices=True)
# # common_lat, idx1_lat, idx2_lat = np.intersect1d(stasource_1.data["lat_1"], stasource_2.data["lat_2"], return_indices=True)
# # idx1 = np.intersect1d(idx1_lon, idx1_lat)
# # idx2 = np.intersect1d(idx2_lon, idx2_lat)
# # common = np.argwhere(common_lon == common_lat)
# # common = common.reshape(-1, 2)
# # common_idx1 = (np.isin(stasource_1.data["lon_1"], stasource_2.data["lon_2"]) & np.isin(stasource_1.data["lat_1"], stasource_2.data["lat_2"]))
# # common_idx2 = (np.isin(stasource_2.data["lon_2"], stasource_1.data["lon_1"]) & np.isin(stasource_2.data["lat_2"], stasource_1.data["lat_1"]))
# # print(common_idx1[0:5])
# # print(stasource_1.data["lon_1"][common_idx1])
# # common_stations = {"lon":stasource_1.data["lon_1"][common_idx1]}
# # ColumnDataSource to hold data for stations in only one folder
# # unique_stations = 



In [106]:
station_1 = pd.DataFrame(stasource_1.data)
station_2 = pd.DataFrame(stasource_2.data)
# Intersect station dataframes based on lon, lat and retain residual velocity components
common = pd.merge(
    station_1, station_2, how="inner", left_on=["lon_1", "lat_1"], right_on=["lon_2", "lat_2"])


combined = pd.concat((station_1[["lon_1", "lat_1"]], station_2[["lon_2", "lat_2"]].rename(columns={"lon_2":"lon_1", "lat_2":"lat_1"}))).drop_duplicates(keep=False)

In [96]:


fig = figure(
    x_range=(0, 360),
    y_range=(-90, 90),
    width=800,
    height=400,
    match_aspect=True,
    # tools=[BoxZoomTool(), ResetTool(), PanTool()],
    # tools=[BoxZoomTool(match_aspect=True), ResetTool(), PanTool()],
    output_backend="webgl",
    toolbar_location="below",
)

# Slip rate color mapper
rescompare_color_mapper = LinearColorMapper(palette=brewer["RdBu"][11], low=-5, high=5)


# Folder 2: residual magnitudes
fig.scatter(
    "lon_c",
    "lat_c",
    source=commonsta,
    marker="circle",
    size="sized_res_mag_diff",
    color={"field": "res_mag_diff", "transform": rescompare_color_mapper},
    line_width=0.5,
    line_color='black',
    line_alpha=1.0,
    visible=True,
)

fig.scatter("lon_u", "lat_u", source=uniquesta, marker='x', size=10, color="red")
show(fig)